# **Distribución de la población por género y rango de edad**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()

# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, '../../db/datos_json')

# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

# Cargar los archivos JSON y crear DataFrames
for archivo in archivos_json:
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())

# Filtrar los nombres que comienzan con "df_", contienen "alfa_q" y "pachuca"
nombres_df_filtrados = [
    nombre for nombre in nombres_variables_globales 
    # Caso de cuando no son las alfa q
    if nombre.startswith("df_poblacion") and ('pachuca' in nombre)

    #if nombre.startswith("df_") and "tulancingo" in nombre    
    #and ("alfa_q" in nombre or "jul_2023" in nombre or "sep_2023" in nombre or "feb_2024" in nombre or "mar_2024" in nombre or "may_2024" in nombre)
]

# Imprimir la lista de DataFrames filtrados
print("Lista de DataFrames filtrados:")
nombres_df_filtrados

In [ ]:
# Iterar sobre cada DataFrame en la lista filtrada
for nombre_df in nombres_df_filtrados:
    # Obtener el DataFrame usando globals()
    df = globals()[nombre_df]
    
    df.rename(columns={'Hombres':'hombres','Mujeres':'mujeres'}, inplace=True)
        # Asignar el DataFrame modificado de nuevo a la variable global
    globals()[nombre_df] = df 

# Imprimir confirmación
print("Columnas renombradas en los DataFrames filtrados.")


In [ ]:
# Crear una lista de DataFrames seleccionados con las columnas específicas
dataframes_list = []
for nombre_df in nombres_df_filtrados:
    # Seleccionar las columnas 'id' y 'categoria'
    segment_df = globals()[nombre_df][['id', 'rango', 'total', 'hombres', 'mujeres']]
    # Añadir el DataFrame a la lista
    dataframes_list.append(segment_df)
dfs = []
for df in dataframes_list:
    dfs.append(df)
dfs = pd.concat(dfs, ignore_index=True)
df = dfs.copy()
df.head()

In [ ]:
## PERCENT HOMBRES
Percentage_H = df[df['Sex'] == 'Hombre']
Percentage_H_total = Percentage_H['Percentage'].sum()

## PERCENT MUJERES
Percentage_M = df[df['Sex'] == 'Mujer']
Percentage_M_total = Percentage_M['Percentage'].sum()


H_total = Percentage_H['Population'].sum()
H_total
M_total = Percentage_M['Population'].sum()
M_total

## Gráficas
### Género de la Población

In [14]:
Population_M_total = Percentage_M['Population'].sum()
Population_H_total = Percentage_H['Population'].sum()
Population_MR = df['Population'].sum()
print("La población se forma por: ", Population_M_total, "Mujeres y ", Population_H_total, "Hombres")
print("En total: ", Population_MR)

La población se forma por:  106271 Mujeres y  96041 Hombres
En total:  202312


In [15]:
df_gen = pd.DataFrame() # Crear el nuevo DataFrame
df_gen['Gen'] = ['Femenino','Masculino']
df_gen['Percentage'] = [Percentage_M_total,Percentage_H_total]
# Mostrar el nuevo DataFrame
print(df_gen)

         Gen  Percentage
0   Femenino   52.528273
1  Masculino   47.471727


In [16]:
colores = ['#3d45c0','#ff4853']
labels = df_gen['Gen']   # Etiquetas de las porciones
values = df_gen['Percentage']  # Valores de las porciones
porcentajes = values
fig = go.Figure(data=[go.Pie(labels=labels, values=values, marker=dict(colors=colores))])
fig.update_traces(textinfo='percent', textposition='inside')
fig.update_layout(
    title='Género de población',  # Título de la gráfica
    #showlegend=False,  # No mostrar la leyenda
    width=630,
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

guardar_grafico_como_html(fig, 'grafica_Genero_pie', carpeta='graficas')

fig.show()

In [17]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=labels,  
    x=values,  
    orientation='h', 
    marker_color=colores,  # Colores de las barras
    text=porcentajes,  # Texto en las barras
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
    texttemplate='%{text:.1f}%',  # Formato del texto (porcentaje con un decimal)
))

# Personalizar el diseño de la gráfica
fig.update_layout(
    title='Género de población',  # Título de la gráfica
    #yaxis=dict(title='Preocupación'),  # Título del eje x
    xaxis=dict(
        title='%', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
        #gridwidth=1,
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'grafica_Genero_Percentage_H', carpeta='graficas')
fig.show()

In [18]:
poblacion_por_genero = df.groupby('Sex')['Population'].sum().reset_index() # Sumar la población por género
values_formatted = poblacion_por_genero['Population'].apply(lambda x: "{:,}".format(x))

labels = poblacion_por_genero['Sex']
values = poblacion_por_genero['Population']

colores = ['#ff4853','#3d45c0']

fig = go.Figure()
fig.add_trace(go.Bar(
    y=labels,  # Etiquetas en el eje y
    x=values,  # Valores en el eje x
    orientation='h',  # Orientación horizontal de las barras
    marker_color=colores,  # Color de las barras
    text=values_formatted,  # Texto formateado en las barras
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    title='Cantidad total de personas por género',  # Título de la gráfica
    xaxis=dict(
        title='Población',  # Título del eje x
        gridcolor='#dddcda',  # Color de las líneas que dividen los rangos del eje Y
    ),
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'grafica_Genero_Personas_H', carpeta='graficas')
fig.show()

______
### Género de la Población por rango de edades

In [19]:
# Crear un DataFrame pivotante para preparar los datos para la visualización
pivot_df = df.pivot(index='Age Range', columns='Sex', values='Population')

colores = ['#ff4853','#3d45c0']

fig = go.Figure()
for col, color in zip(pivot_df.columns, colores):
    fig.add_trace(go.Bar(
        y=pivot_df.index,
        x=pivot_df[col],
        name=col,
        orientation='h',
        marker=dict(color=color),
    ))
fig.update_layout(
    title='Distribución de género por rango de edades',
    xaxis=dict(title='Población',gridcolor='#dddcda'),
    yaxis=dict(title='Rango de edades'),
    barmode='stack',  # Barras apiladas
    plot_bgcolor='rgba(0,0,0,0)',
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'grafica_Genero_RangoEdadesGenero_H', carpeta='graficas')

fig.show()

In [20]:
# Crear un DataFrame pivotante para preparar los datos para la visualización
pivot_df = df.pivot(index='Age Range', columns='Sex', values='Population')

colores = ['#ff4853', '#3d45c0']

fig = go.Figure()
for col, color in zip(pivot_df.columns, colores):
    fig.add_trace(go.Bar(
        y=pivot_df.index,
        x=pivot_df[col],
        name=col,
        orientation='h',
        marker=dict(color=color),
    ))
fig.update_layout(
    title='Distribución de género por rango de edades',
    xaxis=dict(title='Población', gridcolor='#dddcda'),
    yaxis=dict(title='Rango de edades'),
    barmode='group',  # Barras agrupadas
    plot_bgcolor='rgba(0,0,0,0)',
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'grafica_Genero_RangoEdadesGenero_H2', carpeta='graficas')
fig.show()


In [21]:
# Obtener los datos para género masculino y femenino
male_data = df[df['Sex'] == 'Hombre']
female_data = df[df['Sex'] == 'Mujer']

# Obtener los rangos de edades y sus índices
age_ranges = df['Age Range'].unique()
# Eliminar la palabra 'años' de los rangos
# Modificar los rangos de edades
age_ranges_modified = []
for age_range in age_ranges:
    if age_range == '85 años o más':
        age_ranges_modified.append('85+')
    else:
        age_ranges_modified.append(age_range.replace(' años', ''))

indices = list(range(len(age_ranges_modified)))

bar_width = 0.35  # Ancho de las barras

fig = go.Figure()

# Agregar barras para género femenino
fig.add_trace(go.Bar(
    y=age_ranges_modified,
    x=-female_data['Population'],  # Coordenadas x invertidas
    text=female_data['Population'],  # Etiquetas de cantidad positiva
    name='Mujeres',
    orientation='h',
    marker=dict(color='#d0e3ff'),
    textposition='outside',  # Posición del texto fuera de la barra
    
))

# Agregar barras para género masculino
fig.add_trace(go.Bar(
    y=age_ranges_modified,
    x=male_data['Population'],
    text=male_data['Population'],  # Etiquetas de cantidad positiva
    name='Hombres',
    orientation='h',
    marker=dict(color='#506fd9'),
    textposition='outside',  # Posición del texto fuera de la barra
))
# Configurar el fondo de la gráfica como transparente
fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)'
})
# Personalizar el diseño de la gráfica
fig.update_layout(
    #title='Distribución de la población MR por género y rango de edad',
    xaxis=dict(title='Población', showgrid=False, ),
    #yaxis=dict(title='Edad'),
    barmode='overlay',  # Barras superpuestas
    plot_bgcolor='rgba(0,0,0,0)',
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'grafica_Genero_RangoEdadesGenero_LR', carpeta='graficas')
fig.show()




_________
### Distribución de edad para cada género

In [22]:
# Agrupar los datos por género y rango de edades y calcular la población total
grouped_data = df.groupby(['Sex', 'Age Range'])['Population'].sum().unstack()
female_data = grouped_data.loc['Mujer']  # Obtener los datos solo para el género femenino
# Calcular los porcentajes de población para cada grupo de edad
percentages = (female_data / female_data.sum()) * 100
# Crear la lista de colores
colors = ['#506fd9'] * len(percentages)  # Todas las proporciones serán de color azul

fig = go.Figure(data=[go.Pie(labels=female_data.index, values=percentages, marker=dict(colors=colors))]) #textinfo='percent'
fig.update_layout(
    title='Distribución de edad en género Femenino',
    title_font_size=20,
    title_pad=dict(t=20),
    width=630,
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'grafica_Genero_RangoEdadesFem_pie', carpeta='graficas')
fig.show()

In [23]:
# Agrupar los datos por género y rango de edades y calcular la población total
grouped_data = df.groupby(['Sex', 'Age Range'])['Population'].sum().unstack()
female_data = grouped_data.loc['Mujer']  # Obtener los datos solo para el género femenino

# Calcular los porcentajes de población para cada grupo de edad
percentages = (female_data / female_data.sum()) * 100

# Crear la lista de colores
colors = ['#3d45c0'] * len(percentages)  # Todas las proporciones serán de color azul

# Crear la gráfica de pastel con los rangos de edades afuera de cada porción
fig = go.Figure(data=[go.Pie(labels=female_data.index, values=percentages, marker=dict(colors=colors), textinfo='label')])

# Personalizar diseño
fig.update_layout(
    title='Distribución de edad en género Femenino',
    title_font_size=20,
    title_pad=dict(t=20),
    width=630,
    showlegend=False  # Ocultar la leyenda
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'grafica_Genero_RangoEdadesFem_pie', carpeta='graficas')
fig.show()


In [24]:
# Agrupar los datos por género y rango de edades y calcular la población total
grouped_data = df.groupby(['Sex', 'Age Range'])['Population'].sum().unstack()
male_data = grouped_data.loc['Hombre']  # Obtener los datos solo para el género Masculino

# Calcular los porcentajes de población para cada grupo de edad
percentages = (male_data / male_data.sum()) * 100

# Crear la lista de colores
colors = ['#506FD9'] * len(percentages)  # Todas las proporciones serán de color azul

# Crear la gráfica de pastel con los rangos de edades afuera de cada porción
fig = go.Figure(data=[go.Pie(labels=male_data.index, values=percentages, marker=dict(colors=colors), textinfo='label')])

# Personalizar diseño
fig.update_layout(
    title='Distribución de edad en género Masculino',
    title_font_size=20,
    title_pad=dict(t=20),
    width=630,
    showlegend=False  # Ocultar la leyenda
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'grafica_Genero_RangoEdadesMale_pie', carpeta='graficas')
fig.show()


__________
### Porción representativa de la población que podría estar interesada a participar en procesos electorales 

In [25]:
# Juntar en df grupos de edad
Adultez_Temprana_H = df.iloc[3:7]
Adultez_Temprana_M = df.iloc[21:25]
Edad_Media_H = df.iloc[7:11]
Edad_Media_M = df.iloc[25:29]
Adultez_Tardia_H = df.iloc[11:18]
Adultez_Tardia_M = df.iloc[29:36]

In [26]:
# Sacar sumas
Percentage_ATemH = Adultez_Temprana_H['Percentage'].sum()
Percentage_ATemM = Adultez_Temprana_M['Percentage'].sum()
Percentage_EMH = Edad_Media_H['Percentage'].sum()
Percentage_EMM = Edad_Media_M['Percentage'].sum()
Percentage_ATarH = Adultez_Tardia_H['Percentage'].sum()
Percentage_ATarM = Adultez_Tardia_M['Percentage'].sum()

In [33]:
df_pob_inter = pd.DataFrame() # Crear el nuevo DataFrame
#  Columnas
df_pob_inter['Gen'] = ['Masculino','Femenino','Masculino','Femenino','Masculino','Femenino']
df_pob_inter['Age_Range'] = ['15 a 34 años','15 a 34 años','35 a 54 años','35 a 54 años','55 o más','55 o más']
df_pob_inter['Percentage'] = [Percentage_ATemH,Percentage_ATemM,Percentage_EMH,Percentage_EMM,Percentage_ATarH,Percentage_ATarM]
# Mostrar el nuevo DataFrame
print(df_pob_inter)

         Gen     Age_Range  Percentage
0  Masculino  15 a 34 años   16.795840
1   Femenino  15 a 34 años   17.951975
2  Masculino  35 a 54 años   13.322492
3   Femenino  35 a 54 años   16.086045
4  Masculino      55 o más    5.814287
5   Femenino      55 o más    7.180493


In [35]:
import plotly.graph_objects as go
import pandas as pd

# Filtrar datos por género
masculino_data = df_pob_inter[df_pob_inter['Gen'] == 'Masculino']
femenino_data = df_pob_inter[df_pob_inter['Gen'] == 'Femenino']

# Crear el gráfico de pastel
fig = go.Figure()

fig.add_trace(go.Pie(labels=masculino_data['Age_Range'], values=masculino_data['Percentage'], name='Masculino', hole=0.4, domain={'x': [0, 0.5]}))
fig.add_trace(go.Pie(labels=femenino_data['Age_Range'], values=femenino_data['Percentage'], name='Femenino', hole=0.4, domain={'x': [0.5, 1]}))

# Actualizar diseño del gráfico
colors = [
    '#506FD9', 
    '#4455b9', 
    '#3e62c5', 
]
fig.update_traces(textposition='inside', textinfo='percent+label', marker=dict(colors=colors),)

# Añadir título
#fig.update_layout(title_text="Distribución de género por rango de edad")

# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'grafica_Generos_pies2', carpeta='graficas')

fig.show()

In [28]:
# Agrupar los datos por género y rango de edades y calcular la población total
grouped_data = df_pob_inter.groupby(['Gen', 'Age_Range'])['Percentage'].sum().unstack()
female_data = grouped_data.loc['Femenino']  # Obtener los datos solo para el género femenino
# Calcular los porcentajes de población para cada grupo de edad
percentages = (female_data / female_data.sum()) * 100

#colors = ['#4C3228', '#5B3E31','#6A4A3A'] 
base_color = '#3d45c0'
colors = [
    base_color,
    '#4455b9',  # Un tono más claro
    '#3e62c5',  # Un tono más oscuro
]
fig = go.Figure(go.Pie(
    labels=female_data.index,
    values=percentages,
    marker=dict(colors=colors),
    textinfo='percent',
))
fig.update_layout(
    title='Distribución de edad en género Femenino',
    title_font_size=17,
    title_pad=dict(t=16),
    width=700,
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'grafica_Genero_RangoEdadesFem_pie2', carpeta='graficas')
fig.show()

In [29]:
# Agrupar los datos por género y rango de edades y calcular la población total
grouped_data = df_pob_inter.groupby(['Gen', 'Age_Range'])['Percentage'].sum().unstack()
male_data = grouped_data.loc['Masculino']  
# Calcular los porcentajes de población para cada grupo de edad
percentages = (male_data / male_data.sum()) * 100

base_color = '#ff4853'
colors = [
    base_color,
    '#ff5762',  # Un tono más claro
    '#ff6571',  # Un tono más oscuro
]

fig = go.Figure(go.Pie(
    labels=male_data.index,
    values=percentages,
    marker=dict(colors=colors),
    textinfo='percent',
))
fig.update_layout(
    title='Distribución de edad en género Masculino',
    title_font_size=17,
    title_pad=dict(t=16),
    width=700,
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'grafica_Genero_RangoEdadesMale_pie2', carpeta='graficas')
fig.show()